## import package

In [1]:
import scipy.io
import numpy as np
import pandas as pd


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, LinearRegression



In [2]:
#visualization
def confusion_heat_map(model, X, y):
    y_fit = model.predict(X)
    import seaborn as sns
    %matplotlibe notebook 
    from matplotlib import pyplot as plt
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y, y_fit)
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=np.unique(y),
                yticklabels=np.unique(y))

    plt.xlabel('true label')
    plt.ylabel('predicted label');
    plt.show()

## load train and test data

In [3]:
df_5 = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_data_train.csv')
#df_3 = pd.read_csv('D:/BCI_Classification/bci_fft_3_data.csv')


X = df_5[['20','21','22','23','40','41','42','43']]
y = df_5['y']
day = df_5['day']

In [ ]:
np.unique(y)

In [ ]:
np.unique(day)


## simple neural network + AutoEncoder

In [4]:
# from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
# from tensorflow.keras.constraints import max_norm
# import os
# import glob
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.constraints import max_norm

from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import (Input, Dense, Concatenate)
from keras.utils import np_utils
  


def autoencoder_model():
    x = Input(name='inputs', shape=(8,), dtype='float32')
    o = x
    
        # Encoder / Decoder
    o = Dense(8, activation='relu', name='dense1')(o)
    dec = Dense(8, activation='softmax', name='dense_dec')(o)
    
    # Print network summary
    Model(inputs=x, outputs=dec).summary()
    
    return Model(inputs=x, outputs=dec)   

    
#     model = models.Sequential() 
#     model.add(layers.Dense(32,activation='relu',input_shape=(8,), name = 'dense1'))
#     model.add(layers.Dense(8,activation='relu'))
#     model.compile(optimizer =keras.optimizers.Adam(lr = 0.001),
#                       loss = 'mse')
    
    return model
    

def FEATURES(model):
    input_ = model.get_layer('inputs').input
    feat = model.get_layer('dense1').output
    model = Model(inputs=[input_],
                      outputs=[feat])
    return model

    
# Function to create model, required for KerasClassifier
def create_model():
    model = models.Sequential() 
    model.add(layers.Flatten(input_shape=(8,)))
    
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(5, activation='softmax'))

    #model compilation
    #Compile and train the model
    model.compile(optimizer ='adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics=['acc'])
    
    return model



Using TensorFlow backend.


In [5]:
def split_train_test(X,y, day, test_size=0.2, random_state=42):
    ## X,y as the pandas series,
    y = y.values
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify = day)

    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=test_size, random_state=random_state)
    
    
    
    
    
    
    
    
    
    
    
    
    return X_train, X_val,X_test, y_train, y_val, y_test

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify = day)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)



y_train = y_train.values
y_test = y_test.values
y_val = y_val.values


noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_val_noisy = X_val + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_val.shape)
# X_train_noisy = np.clip(X_train_noisy, 0., 1.)
# X_test_noisy = np.clip(X_test_noisy, 0., 1.)

autoencoder = autoencoder_model()
autoencoder.compile(optimizer ='adam', loss = 'mse')
history_AE = autoencoder.fit(X_train_noisy, X_train, verbose = 0, epochs=100, validation_data=(X_val_noisy, X_val))

_model = FEATURES(autoencoder)
features_train = _model.predict(X_train)
features_val = _model.predict(X_val)
features_test = _model.predict(X_test)
print(features_train.shape, ' train samples shape')
print(features_val.shape, ' val samples shape')


# # checkpoint
filepath="D://BCI_Classification//AEmodel_8day_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(features_train, y_train, epochs=200, validation_data=(features_val, y_val),
                    callbacks=callbacks_list,verbose = 0)
    
#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(features_test,y_test, verbose = 2)

print('Model evaluation:{}'.format(test_acc))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label ='val_loss' )
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()



y_fit = model.predict_classes(features_test)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_fit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_test),
            yticklabels=np.unique(y_test))

plt.xlabel('true label')
plt.ylabel('predicted label');
plt.show()

## cross day prediction

In [6]:
from sklearn.model_selection import KFold
for train_day, predict_day in  KFold(8).split([0,1,2,3,4,5,6,7]):
    day_value = day.values
    day_predict_idx = np.unique(day_value)[predict_day]
    print(day_predict_idx)
    train_list = (day_value != day_predict_idx)

    #training set
    X_train = X.iloc[train_list==True , :]
    y_train = y[train_list==True]

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify = y_train)

    #test set
    X_test= X.iloc[train_list == False, :]
    y_test= y[train_list==False]


    y_train = y_train.values
    y_test = y_test.values
    y_val = y_val.values


    noise_factor = 0.5
    X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
    X_val_noisy = X_val + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_val.shape)
    # X_train_noisy = np.clip(X_train_noisy, 0., 1.)
    # X_test_noisy = np.clip(X_test_noisy, 0., 1.)

    autoencoder = autoencoder_model()
    autoencoder.compile(optimizer ='adam', loss = 'mse')
    history_AE = autoencoder.fit(X_train_noisy, X_train, verbose = 0, epochs=100, validation_data=(X_val_noisy, X_val))

    _model = FEATURES(autoencoder)
    features_train = _model.predict(X_train)
    features_val = _model.predict(X_val)
    features_test = _model.predict(X_test)
    print(features_train.shape, ' train samples shape')
    print(features_val.shape, ' val samples shape')


    # # checkpoint
    filepath="D://BCI_Classification//AEmodel_crossday_weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    #Compile and train the model
    model = create_model()
    history = model.fit(features_train, y_train, epochs=200, validation_data=(features_val, y_val),
                        callbacks=callbacks_list,verbose = 0)

    #evaluation
    model.load_weights(filepath)
    test_loss,test_acc = model.evaluate(features_test,y_test, verbose = 2)

    print('Model evaluation:{}'.format(test_acc))
    
    from matplotlib import pyplot as plt
    plt.plot(history.history['acc'], label='accuracy')
    plt.plot(history.history['val_acc'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'], label = 'loss')
    plt.plot(history.history['val_loss'], label ='val_loss' )
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()



    y_fit = model.predict_classes(features_test)
    import seaborn as sns
    from matplotlib import pyplot as plt
    from sklearn.metrics import confusion_matrix
    mat = confusion_matrix(y_test, y_fit)
    sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
                xticklabels=np.unique(y_test),
                yticklabels=np.unique(y_test))

    plt.xlabel('true label')
    plt.ylabel('predicted label');
    plt.show()

[0]
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 8)                 0         
_________________________________________________________________
dense1 (Dense)               (None, 8)                 72        
_________________________________________________________________
dense_dec (Dense)            (None, 8)                 72        
Total params: 144
Trainable params: 144
Non-trainable params: 0
_________________________________________________________________
(6365, 8)  train samples shape
(1592, 8)  val samples shape
27/27 - 0s - loss: 3.0693 - acc: 0.4105
Model evaluation:0.410526305437088


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


c:\users\owner\miniconda3\envs\tfeeg\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


...

<Figure size 640x480 with 1 Axes>

[7]
Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 8)                 0         
_________________________________________________________________
dense1 (Dense)               (None, 8)                 72        
_________________________________________________________________
dense_dec (Dense)            (None, 8)                 72        
Total params: 144
Trainable params: 144
Non-trainable params: 0
_________________________________________________________________
(6032, 8)  train samples shape
(1509, 8)  val samples shape
40/40 - 0s - loss: 6.0731 - acc: 0.3643
Model evaluation:0.364280104637146


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label ='val_loss' )
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()



y_fit = model.predict_classes(features_test)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_fit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_test),
            yticklabels=np.unique(y_test))

plt.xlabel('true label')
plt.ylabel('predicted label');
plt.show()

## same day prediction

In [ ]:
X_train = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_sameday_data_train.csv')
X_test = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_sameday_data_test.csv')


y_train = X_train['y']
y_test = X_test['y']

#alpha only
X_train = X_train[['20','21','22','23','40','41','42','43']]
X_test = X_test[['20','21','22','23','40','41','42','43']]



from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.4, random_state=2, stratify = y_train)



# X_train = X_train.values
# X_test = X_test.values
# X_val = X_val.values


y_train = y_train.values
y_test = y_test.values
y_val = y_val.values


noise_factor = 0.5
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_val_noisy = X_val + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_val.shape)
X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

autoencoder = autoencoder_model()
autoencoder.compile(optimizer ='adam', loss = 'mse')
history_AE = autoencoder.fit(X_train_noisy, X_train, verbose = 0, epochs=100, validation_data=(X_val_noisy, X_val))

_model = FEATURES(autoencoder)
features_train = _model.predict(X_train)
features_val = _model.predict(X_val)
features_test = _model.predict(X_test)
print(features_train.shape, ' train samples shape')
print(features_val.shape, ' val samples shape')


# # checkpoint
filepath="D://BCI_Classification//AEmodel_sameday_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(features_train, y_train, epochs=200, validation_data=(features_val, y_val),
                    callbacks=callbacks_list,verbose = 0)
    
#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(features_test,y_test, verbose = 2)

print('Model evaluation:{}'.format(test_acc))



In [ ]:
X_train = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_step250_sameday_data_train.csv')
X_test = pd.read_csv('D:/BCI_Classification/bci_fft_5freq_step250_sameday_data_test.csv')


y_train = X_train['y']
y_test = X_test['y']

#alpha only
X_train = X_train[['20','21','22','23','40','41','42','43']]
X_test = X_test[['20','21','22','23','40','41','42','43']]


X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=2, stratify = y_train)

# X_train, X_test, y_train, y_test = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=2,stratify = y_train )


X_train = np.expand_dims(X_train,axis=1)
X_test = np.expand_dims(X_test,axis=1)
X_val = np.expand_dims(X_val,axis=1)

y_train = y_train.values
y_test = y_test.values
y_val = y_val.values

# # checkpoint
filepath="D://BCI_Classification//model_sameday_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(X_train, y_train,epochs=200, validation_data=(X_val, y_val), 
                    callbacks=callbacks_list,verbose = 0)
    
#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(X_test,y_test, verbose = 2)

print('Model evaluation:{}'.format(test_acc))



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=2, stratify = y)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=2,stratify = y_train )


X_train = np.expand_dims(X_train,axis=1)
X_test = np.expand_dims(X_test,axis=1)
X_val = np.expand_dims(X_val,axis=1)

y_train = y_train.values - 2
y_test = y_test.values - 2
y_val = y_val.values - 2 

# checkpoint
filepath="D://BCI_Classification//model_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(X_train, y_train,epochs=200, validation_data=(X_val, y_val), callbacks=callbacks_list,verbose = 1)
    
#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(X_test,y_test, verbose = 2)

print('Model evaluation:{}'.format(test_acc))



In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label ='val_loss' )
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()



y_fit = model.predict_classes(X_test)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, y_fit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=np.unique(y_test),
            yticklabels=np.unique(y_test))

plt.xlabel('true label')
plt.ylabel('predicted label');
plt.show()

### individual day calibration

In [ ]:
for train_day, predict_day in  KFold(5).split([0,1,2,3,4]):
    print(predict_day)

In [ ]:
# from sklearn.model_selection import KFold

# day_value = day.values

# for train_day, predict_day in  KFold(5).split([0,1,2,3,4]):
predict_day = [1]
    
day_predict_idx = np.unique(day_value)[predict_day]
print(day_predict_idx)
train_list = (day_value != day_predict_idx)

#training set
X_train = X.iloc[train_list==True , :]
y_train = y[train_list==True]

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=2, stratify = y_train)

#test set
X_test= X.iloc[train_list == False, :]
y_test= y[train_list==False]

X_train = np.expand_dims(X_train,axis=1)
X_test = np.expand_dims(X_test,axis=1)
X_val = np.expand_dims(X_val,axis=1)

y_train = y_train.values - 2
y_test = y_test.values - 2
y_val = y_val.values - 2 

# checkpoint
filepath="D://BCI_Classification//day_"+str(day_predict_idx[0])+"_model_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
model = create_model()
history = model.fit(X_train, y_train,epochs=100, validation_data=(X_val, y_val), callbacks=callbacks_list)

#evaluation
model.load_weights(filepath)
test_loss,test_acc = model.evaluate(X_test,y_test, verbose = 2)

print('Model evaluation:{}'.format(test_acc))



X_retrain, X_reval, y_retrain, y_reval = train_test_split(
    X_test, y_test, test_size=0.2, random_state=2, stratify = y_test)

X_retrain, X_retest, y_retrain, y_retest = train_test_split(
    X_retrain, y_retrain, test_size=0.8, random_state=2, stratify = y_retrain)



# retrain checkpoint
refilepath="D://BCI_Classification//day_"+str(day_predict_idx[0])+"_remodel_weights.hdf5"
checkpoint = ModelCheckpoint(refilepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#Compile and train the model
remodel = create_model()
remodel.load_weights(filepath)
history = remodel.fit(X_retrain, y_retrain, epochs=100, validation_data=(X_val, y_val), callbacks=callbacks_list)

#evaluation
remodel.load_weights(refilepath)
retest_loss,retest_acc = model.evaluate(X_retest,y_retest, verbose = 2)
print('Model evaluation:{}'.format(retest_acc))





In [ ]:
for i in range(1,6):
    filepath="D://BCI_Classification//day_"+str(i)+"_model_weights.hdf5"
    #evaluation
    model.load_weights(filepath)
    
    X_train = X[day!=i]
    y_train = y[day!=i]
    X_train = np.expand_dims(X_train,axis=1)
    y_train = y_train.values - 2
    
    

    X_test = X[day==i]
    y_test = y[day==i]
    X_test = np.expand_dims(X_test,axis=1)
    y_test = y_test.values - 2
    
    
    train_loss,train_acc = model.evaluate(X_train,y_train, verbose = 0)
    test_loss,test_acc = model.evaluate(X_test,y_test, verbose = 0)
    print('Predict day:{}'.format(i))
    print('training set:{}'.format(train_acc))
    print('test set:{}'.format(test_acc))
    print()

In [ ]:
from sklearn.model_selection import KFold

day_value = day.values

for train_day, predict_day in  KFold(5).split([0,1,2,3,4]):
    
    day_predict_idx = np.unique(day_value)[predict_day]
    print(day_predict_idx)
    train_list = (day_value != day_predict_idx)
    
    #training set
    X_train = X.iloc[train_list==True , :]
    y_train = y[train_list==True]
       
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=2, stratify = y)
    
    #test set
    X_test= X.iloc[train_list == False, :]
    y_test= y[train_list==False]

    X_train = np.expand_dims(X_train,axis=1)
    X_test = np.expand_dims(X_test,axis=1)
    X_val = np.expand_dims(X_val,axis=1)

    y_train = y_train.values - 2
    y_test = y_test.values - 2
    y_val = y_val.values - 2 

    # checkpoint
    filepath="D://BCI_Classification//day_"+str(day_predict_idx[0])+"_model_weights.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    #Compile and train the model
    model = create_model()
    history = model.fit(X_train, y_train,epochs=100, validation_data=(X_val, y_val), callbacks=callbacks_list)

    #evaluation
    model.load_weights(filepath)
    test_loss,test_acc = model.evaluate(X_test,y_test)

    print('Model evaluation:{}'.format(test_acc))

